# Compiling Circuits for EeroQ via Qiskit

## Import Requirements

This tutorial will showcase how to compile a circuit for EeroQ hardware using the ```qiskit-superstaq``` client. 

In [2]:
# Required imports
try:
    import qiskit
    import qiskit_superstaq as qss
except ImportError:
    print("Installing qiskit-superstaq...")
    %pip install --quiet 'qiskit-superstaq[examples]'
    print("Installed qiskit-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import qiskit
    import qiskit_superstaq as qss

# Optional imports
import numpy as np
import os  # Used if setting a token as an environment variable

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


To interface Superstaq via Qiskit, we must first instantiate a provider in `qiskit-superstaq` with `SuperstaqProvider()`. We then supply a Superstaq API token (or key) by either providing the API token as an argument of `qss.SuperstaqProvider()` or by setting it as an environment variable (see more details [here](https://superstaq.readthedocs.io/en/latest/get_started/basics/basics_qss.html#Set-up-access-to-Superstaq%E2%80%99s-API))

In [3]:
service = qss.superstaq_provider.SuperstaqProvider()

# EeroQ Gates

In [4]:
dd_gate = qss.DDGate(1)
dd_gate

qss.DDGate(1)

In [5]:
qc = qiskit.QuantumCircuit(2)
qc.append(dd_gate, [0, 1])
qc.draw()

┌────────┐
q_0: ┤0       ├
     │  Dd(1) │
q_1: ┤1       ├
     └────────┘

Below is the EeroQ protocol for a CZ Gate.

![title](images/DD_gate.png)

In [6]:
def count_gates(qc: qiskit.QuantumCircuit):
    gate_count = { qubit: 0 for qubit in qc.qubits }
    for gate in qc.data:
        for qubit in gate.qubits:
            gate_count[qubit] += 1
    return gate_count

def remove_idle_wires(qc: qiskit.QuantumCircuit):
    qc_out = qc.copy()
    gate_count = count_gates(qc_out)
    for qubit, count in gate_count.items():
        if count == 0:
            qc_out.qubits.remove(qubit)
    return qc_out

In [7]:
qc = qiskit.QuantumCircuit(2)
qc.cz(0, 1)
qc.measure_all()
backend = service.get_backend("eeroq_wonderlake_qpu")
compiled_circuit = backend.compile(qc).circuit
remove_idle_wires(compiled_circuit).draw()

░                                 ░       ┌─┐   
 0+: ──────░─────────────────────────────────░───────┤M├───
     ┌───┐ ░ ┌──────────┐ ┌───┐ ┌──────────┐ ░       └╥┘┌─┐
 0-: ┤ X ├─░─┤0         ├─┤ S ├─┤0         ├─░────────╫─┤M├
     └───┘ ░ │          │ ├───┤ │          │ ░ ┌─┐    ║ └╥┘
 1+: ──────░─┤  Dd(π/4) ├─┤ Z ├─┤  Dd(π/4) ├─░─┤M├────╫──╫─
     ┌───┐ ░ │          │┌┴───┴┐│          │ ░ └╥┘┌─┐ ║  ║ 
 1-: ┤ X ├─░─┤1         ├┤ Sdg ├┤1         ├─░──╫─┤M├─╫──╫─
     └───┘ ░ └──────────┘└─────┘└──────────┘ ░  ║ └╥┘ ║  ║ 
c: 4/═══════════════════════════════════════════╩══╩══╩══╩═
                                                2  3  0  1

In [8]:
job = backend.run(qc, shots = 10, method="dry-run")

In [9]:
print(job.result().get_counts())

{'00': 10}


# Circuit Compilation

In [16]:
from qiskit.circuit.random import random_circuit
qc = random_circuit(2, 3, measure=False)
qc.draw()

┌────────────┐          ┌───┐        ┌────────────┐
q_0: ───┤ Rx(1.5533) ├──────────┤ H ├────────┤ Ry(3.9368) ├
     ┌──┴────────────┴──┐┌──────┴───┴───────┐└────────────┘
q_1: ┤ R(4.6905,3.4663) ├┤ R(4.4294,2.0975) ├──────────────
     └──────────────────┘└──────────────────┘

In [17]:
backend = service.get_backend("eeroq_wonderlake_qpu")

In [18]:
cc = backend.compile(qc).circuit

In [19]:
remove_idle_wires(cc).draw()

░ ┌─────────────┐ ░ ┌──────────────┐ ┌────────────┐
0+: ──────░─┤ Rz(-0.7757) ├─░─┤0             ├─┤ Rz(3.1538) ├
    ┌───┐ ░ └─────────────┘ ░ │  Dd(-1.5583) │ └────────────┘
0-: ┤ X ├─░─────────────────░─┤1             ├───────────────
    └───┘ ░  ┌────────────┐ ░ ├──────────────┤┌─────────────┐
1+: ──────░──┤ Rz(1.5387) ├─░─┤0             ├┤ Rz(0.60522) ├
    ┌───┐ ░  └────────────┘ ░ │  Dd(-1.7585) │└─────────────┘
1-: ┤ X ├─░─────────────────░─┤1             ├───────────────
    └───┘ ░                 ░ └──────────────┘

# Circuit Simulation

In [20]:
from qiskit.circuit.random import random_circuit
qc = random_circuit(2, 3, measure=True)
qc.draw()

┌────────────┐     ┌────────────┐┌─┐
q_0: ┤ Rx(2.2136) ├──■──┤ Rx(4.5827) ├┤M├
     └─────┬──────┘┌─┴─┐└────┬─┬─────┘└╥┘
q_1: ──────■───────┤ S ├─────┤M├───────╫─
                   └───┘     └╥┘       ║ 
c: 2/═════════════════════════╩════════╩═
                              1        0

In [21]:
job = backend.run(qc, shots=100, method="noise-sim", error_rate=0.01)

In [22]:
print(job.result().get_counts())

{'01': 52, '02': 2, '00': 32, '22': 3, '21': 1, '20': 1}


A measurement value of 2 refers to an out-of-codespace error.